In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: white;"><b style="color:#228B22;">(WiDS) Datathon 2022 on Kaggle!</b></h1></center>

The Women in Data Science (WiDS) Datathon 2022 is Now Live on Kaggle!

"The 5th Annual Women in Data Science (WiDS) Datathon is now open on Kaggle, and the competition has begun! This annual datathon encourages women to hone their skills through a  data science challenge focused on social impact.



![](https://icme.stanford.edu/sites/g/files/sbiybj17116/files/styles/card_1900x950/public/media/image/wids_worldwide_2022_registration_1200x628_0.png?h=2240b8e9&itok=RuTrJvuk)icme.stanford.edu

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook


# PyTesseract and Tika-Python for OCR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import PIL
import os
from os import walk
from shutil import copytree, ignore_patterns
from collections import Counter
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from wand.image import Image as Img
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
#mueller_report = pd.read_csv('../input/data-science-cheat-sheets/Interview Questions/AI Questions.pdf') # one row per line

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

# Define helper function for plotting word clouds
def wordCloudFunction(df,column,numWords):
    # adapted from https://www.kaggle.com/benhamner/most-common-forum-topic-words
    topic_words = [ z.lower() for y in
                       [ x.split() for x in df[column] if isinstance(x, str)]
                       for z in y]
    word_count_dict = dict(Counter(topic_words))
    popular_words = sorted(word_count_dict, key = word_count_dict.get, reverse = True)
    popular_words_nonstop = [w for w in popular_words if w not in stopwords.words("english")]
    word_string=str(popular_words_nonstop)
    wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white',
                          max_words=numWords,
                          width=1000,height=1000,
                         ).generate(word_string)
    plt.clf()
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

# Define helper function for plotting word bar graphs
def wordBarGraphFunction(df,column,title):
    # adapted from https://www.kaggle.com/benhamner/most-common-forum-topic-words
    topic_words = [ z.lower() for y in
                       [ x.split() for x in df[column] if isinstance(x, str)]
                       for z in y]
    word_count_dict = dict(Counter(topic_words))
    popular_words = sorted(word_count_dict, key = word_count_dict.get, reverse = True)
    popular_words_nonstop = [w for w in popular_words if w not in stopwords.words("english")]
    plt.barh(range(50), [word_count_dict[w] for w in reversed(popular_words_nonstop[0:50])])
    plt.yticks([x + 0.5 for x in range(50)], reversed(popular_words_nonstop[0:50]))
    plt.title(title)
    plt.show()

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

# Preview the data folder
inputFolder = '../input/'
for root, directories, filenames in os.walk(inputFolder):
    for filename in filenames: 
        print(os.path.join(root,filename))
        
# Move data to folder with read/write access
outputFolder = '/kaggle/working/pdfs/'
shutil.copytree(inputFolder,outputFolder,ignore=ignore_patterns('*.db'))
for root, directories, filenames in os.walk(outputFolder, topdown=False):
    for file in filenames:
        try:
            shutil.move(os.path.join(root, file), outputFolder)
        except OSError:
            pass
print(os.listdir(outputFolder))

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

# Look at page 1
pdf = os.path.join(outputFolder,'dataset_description.pdf[1]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/dataset_description.jpg') # intro page to preview later

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

# Parse a PDF file and convert it to CSV using PyTesseract
import pytesseract
pdfimage = Image.open('/kaggle/working/dataset_description.jpg')
text = pytesseract.image_to_string(pdfimage)  
df = pd.DataFrame([text.split('\n')])

In [ ]:
# Code by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

# Plot WordCloud of page 1
plt.figure(figsize=(10,10))
wordCloudFunction(df.T,0,10000000)
plt.figure(figsize=(10,10))
wordBarGraphFunction(df.T,0,"Most Common Words on Page 1 of L'Important C'est WiDS")

In [ ]:
# Parse a PDF file and convert it to CSV using Tika-Python
!pip install tika
import tika
from tika import parser
tika.initVM()
parsed = parser.from_file('/kaggle/working/dataset_description.jpg') 
text = parsed["content"]
df = pd.DataFrame([text.split('\n')])
df.drop(df.iloc[:, 1:46], inplace=True, axis=1)

In [ ]:
# Convert PDF to JPG and then convert JPG to CSV
# I will do this for Pages 289 to 291 but
# Eventually I should loop through the entire document

# PDF to JPG for p1
pdf = os.path.join(outputFolder,'dataset_description.pdf[1]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/dataset_description.jpg')
pdfimage1 = Image.open('/kaggle/working/dataset_description.jpg')

In [ ]:
# PDF to JPG for p2
pdf = os.path.join(outputFolder,'dataset_description.pdf[2]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/dataset_description.jpg')
pdfimage2 = Image.open('/kaggle/working/dataset_description.jpg')

# PDF to JPG for p1  #That's going to appear write pdf[0] previous number from what is intended to show
pdf = os.path.join(outputFolder,'dataset_description.pdf[0]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/dataset_description.jpg')
pdfimage0 = Image.open('/kaggle/working/dataset_description.jpg')

In [ ]:
# Parse a PDF file and convert it to CSV using PyTesseract (p1)
text = pytesseract.image_to_string(pdfimage0)
df = pd.DataFrame([text.split('\n')])
df.drop(df.iloc[:, 27:], inplace=True, axis=1)
df.drop(df.iloc[:, :3], inplace=True, axis=1)
df.columns = range(df.shape[1])

In [ ]:
# Parse a PDF file and convert it to CSV using Tika-Python (p1-2)
tika.initVM()
parsed = parser.from_file('/kaggle/working/dataset_description.jpg')
parsed2 = parser.from_file('/kaggle/working/dataset_description.jpg')

text = parsed["content"]
df2 = pd.DataFrame([text.split('\n')])
df2.drop(df2.iloc[:, 1:50], inplace=True, axis=1)
df2.drop(df2.iloc[:, 26:], inplace=True, axis=1)
df2.columns = range(df2.shape[1])

text = parsed2["content"]
df3 = pd.DataFrame([text.split('\n')])
df3.drop(df3.iloc[:, :50], inplace=True, axis=1)
df3.drop(df3.iloc[:, 22:], inplace=True, axis=1)
df3.columns = range(df3.shape[1])

dfcombined = pd.concat([df, df2, df3]) # combine pages 1-2

I chose pg 1, so I wrote page 0 below it's 1.
The original crop values were 1240 and 1300. Changing to 300 (upper) and 900 (bottom) I got better perspective of the page.

In [ ]:
#Explore page 1 -   Here we have 3 pages  
w, h = pdfimage1.size # crop image
pdfimage0.crop((0, 300, w, h-900)) # display exerpt of PDF

<h1><span class="label label-default" style="background-color:white;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#228B22; padding:10px">Sorry Stanford codes should have a Soundtrack</span></h1><br>

<iframe width="707" height="530" src="https://www.youtube.com/embed/JhCOxnSPFLc" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

L'important c'est la rose -Gilbert Bécaud- (avec sous-titres)

https://www.youtube.com/watch?v=JhCOxnSPFLc

In [ ]:
# Convert PDF to JPG and then convert JPG to CSV

# Eventually I should loop through the entire document

# PDF to JPG for p2. Write pdf[1] previous page
pdf = os.path.join(outputFolder,'dataset_description.pdf[1]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/dataset_description.jpg')
pdfimage1 = Image.open('/kaggle/working/dataset_description.jpg')

In [ ]:
#Explore page 2 - Mueller Report. Here, there are 3 pages Write the previous page to get what you want
w, h = pdfimage1.size # crop image
pdfimage1.crop((0, 300, w, h-900)) # display exerpt of PDF

In [ ]:
# Convert PDF to JPG and then convert JPG to CSV

# Eventually I should loop through the entire document

# PDF to JPG for p3
pdf = os.path.join(outputFolder,'dataset_description.pdf[2]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/dataset_description.jpg')
pdfimage2 = Image.open('/kaggle/working/dataset_description.jpg')

In [ ]:
#Explore page 3 - Mueller Report. Here, there are 3 pages 
w, h = pdfimage2.size # crop image
pdfimage2.crop((0, 200, w, h-900)) # display exerpt of PDF

In [ ]:
# Pages 1, 2, 3,  
dfcombined.head() # preview csv of 5-11

In [ ]:
# Clean up the notebook
!apt-get install zip # install zip
!zip -r pdfs.zip /kaggle/working/pdfs/ # zip up a few files
!rm -rf pdfs/* # remove everything else

#Acknowledgement:

Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

Ben Hamner https://www.kaggle.com/benhamner/most-common-forum-topic-words

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: white;"><b style="color:#228B22;">Take this Kaggle Challenge</b></h1></center>

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: white;"><b style="color:#228B22;">L'Important C'est WiDS</b></h1></center>